# Homework #3: PCA/Hyperparameter/CV
Data source: https://www.kaggle.com/c/GiveMeSomeCredit


邓廷钦 Deng TingQin (Quintin) 2301212321

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('data/cs-training.csv', index_col=0)
data.fillna(data.mean(), inplace=True)
data.describe()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
count,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,1.500000e+05,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000
mean,0.066840,6.048438,52.295207,0.421033,353.005076,6.670221e+03,8.452760,0.265973,1.018240,0.240387,0.757222
std,0.249746,249.755371,14.771866,4.192781,2037.818523,1.288045e+04,5.145951,4.169304,1.129771,4.155179,1.100404
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.029867,41.000000,0.000000,0.175074,3.903000e+03,5.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.154181,52.000000,0.000000,0.366508,6.600000e+03,8.000000,0.000000,1.000000,0.000000,0.000000
75%,0.000000,0.559046,63.000000,0.000000,0.868254,7.400000e+03,11.000000,0.000000,2.000000,0.000000,1.000000
max,1.000000,50708.000000,109.000000,98.000000,329664.000000,3.008750e+06,58.000000,98.000000,54.000000,98.000000,20.000000


In [2]:
from sklearn.model_selection import train_test_split

trainData = data.values
X, y = trainData[:, 1:], trainData[:, 0].astype('int')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

In [3]:
import sklearn.preprocessing as skpre

stdsc = skpre.StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
print(X_train_std.shape)
X_test_std = stdsc.transform(X_test)
print(X_test_std.shape)

(105000, 10)
(45000, 10)


## PCA using pipeline

In [4]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline

In [5]:
# use the first 3 main components

# lr
pipe_lr = make_pipeline(PCA(n_components=3),
                         LogisticRegression(random_state=1, solver='lbfgs'))
pipe_lr.fit(X_train_std, y_train)
y_pred = pipe_lr.predict(X_test_std)
print('Test Accuracy for LR: %.3f' % pipe_lr.score(X_test_std, y_test))

# svm 
pipe_svm = make_pipeline(PCA(n_components=3),
                         SVC())
pipe_svm.fit(X_train_std, y_train)
y_pred = pipe_svm.predict(X_test_std)
print('Test Accuracy for SVM: %.3f' % pipe_svm.score(X_test_std, y_test))

# dt
pipe_dt = make_pipeline(PCA(n_components=3),
                         DecisionTreeClassifier())
pipe_dt.fit(X_train_std, y_train)
y_pred = pipe_dt.predict(X_test_std)
print('Test Accuracy for Decision Tree: %.3f' % pipe_dt.score(X_test_std, y_test))

Test Accuracy for LR: 0.933
Test Accuracy for SVM: 0.934
Test Accuracy for Decision Tree: 0.894


## 5 fold Cross-Validation

In [6]:
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=5).split(X_train_std, y_train)
for k, (train, test) in enumerate(kfold):
    pipe_lr.fit(X_train_std[train], y_train[train])
    score = pipe_lr.score(X_train_std[test], y_train[test])
    print('Fold: %2d, Class Number: %s, Acc: %.3f' % (k+1, np.bincount(y_train[train]), score))

Fold:  1, Class Number: [78385  5615], Acc: 0.933
Fold:  2, Class Number: [78385  5615], Acc: 0.933
Fold:  3, Class Number: [78386  5614], Acc: 0.933
Fold:  4, Class Number: [78386  5614], Acc: 0.933
Fold:  5, Class Number: [78386  5614], Acc: 0.933


## Grid search

In [7]:
from sklearn.model_selection import GridSearchCV

# gridsearchCV for choosing the best parameters of dt
param_grid = {'max_depth': [2, 3, 4, 5, 6],
              'min_samples_leaf': [2, 3, 4],
              'criterion': ['gini', 'entropy']}

gs = GridSearchCV(estimator=DecisionTreeClassifier(random_state=0),
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=5,
                  verbose=1)


gs = gs.fit(X_train_std, y_train)
print(gs.best_score_)
print(gs.best_params_)
clf = gs.best_estimator_
clf

Fitting 5 folds for each of 30 candidates, totalling 150 fits


0.9359999999999999
{'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 2}


DecisionTreeClassifier(max_depth=5, min_samples_leaf=2, random_state=0)